In [1]:
import os
import sys
import pickle
import numpy as np
from scipy import stats
from sklearn.metrics import mean_squared_error

sys.path.append(os.path.dirname(os.path.dirname(os.getcwd())))
from bayesmf.scripts.utils.model import run_kfold_xval
from bayesmf.scripts.utils.dataset import make_insilico_dataset, make_downsampled_dataset
from bayesmf.models.cmf import CMF, StochasticCMF

%reload_ext autoreload
%autoreload 2

import seaborn as sns
import matplotlib.pyplot as plt